# 02_baseline_transformer_beto — Binario A/D

**Objetivo:** baseline con **transformer en español** (*roberta-bne* o equivalente) para contrastar con TF‑IDF y reglas.  
**Justificación:** modelos preentrenados capturan semántica y contexto; con preprocesamiento conservador suelen superar a métodos clásicos cuando hay suficiente señal.


In [1]:
# ===============================================================
# Setup: Paths, Imports, y Utilidades Compartidas
# ===============================================================

from pathlib import Path
import pandas as pd
import re, unicodedata, os

# Intentar importar utilidades compartidas
try:
    import sys
    sys.path.insert(0, str(Path.cwd()))
    from utils_shared import setup_paths, guess_text_col, guess_label_col, normalize_label
    print("[INFO] Utilizando utils_shared.py")
    
    # Setup de paths centralizado
    paths = setup_paths()
    BASE_PATH = paths['BASE_PATH']
    DATA_PATH = paths['DATA_PATH']
    SPLITS_PATH = paths['SPLITS_PATH']
    
    # Usar funciones centralizadas
    _guess_text_col = guess_text_col
    _guess_label_col = guess_label_col
    _norm_label_bin = normalize_label
    
except ImportError:
    print("[WARNING] utils_shared.py no encontrado, usando funciones locales")
    
    # Setup manual de paths
    BASE_PATH = Path.cwd()
    if BASE_PATH.name == "notebooks":
        BASE_PATH = BASE_PATH.parent
    
    DATA_PATH = BASE_PATH / "data"
    SPLITS_PATH = DATA_PATH / "splits"
    
    DATA_PATH.mkdir(exist_ok=True)
    
    # Funciones helper locales
    def _guess_text_col(df):
        for c in ["texto", "text", "comment", "comentario"]:
            if c in df.columns:
                return c
        return df.columns[0]
    
    def _guess_label_col(df):
        for c in ["etiqueta", "label", "category"]:
            if c in df.columns:
                return c
        return df.columns[1] if len(df.columns) > 1 else df.columns[-1]
    
    def _norm_label_bin(s):
        if pd.isna(s): 
            return ""
        s = str(s).strip().lower()
        s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
        return {'depresivo': 'depresion'}.get(s, s)

# Validar existencia de splits
if not SPLITS_PATH.exists():
    raise FileNotFoundError(
        f"[ERROR] Splits no encontrados en {SPLITS_PATH}\n"
        f"        Debes ejecutar primero: 02_create_splits.ipynb"
    )

print(f"[INFO] Paths configurados:")
print(f"  BASE_PATH:   {BASE_PATH}")
print(f"  DATA_PATH:   {DATA_PATH}")
print(f"  SPLITS_PATH: {SPLITS_PATH}")

# Columnas esperadas en dataset_base.csv
TEXT_COL = "texto"
LABEL_COL = "etiqueta"

[INFO] Utilizando utils_shared.py
[INFO] Paths configurados:
  BASE_PATH:   /Users/manuelnunez/Projects/psych-phenotyping-paraguay
  DATA_PATH:   /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data
  SPLITS_PATH: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits


## 1) Carga y preprocesamiento **conservador** (preserva tildes/casing)

In [2]:
# ===============================================================
# Carga de Datos y Preprocesamiento CONSERVADOR (BETO/Transformer)
# ===============================================================
#
# ESTRATEGIA DE PREPROCESAMIENTO: CONSERVADORA (mínimo)
#
# ¿Por qué preprocesamiento conservador/mínimo?
#
# 1. **Los transformers están preentrenados con texto "natural"**:
#    - BETO/RoBERTa se entrenaron con Wikipedia, noticias, web en español
#    - Ese texto tiene mayúsculas, tildes, puntuación original
#    - Normalizar agresivamente = salirse de la distribución de entrenamiento
#
# 2. **Tokenización BPE maneja variaciones**:
#    - "Depresión" y "depresión" → mismo subtokens (el tokenizer lo normaliza)
#    - El modelo aprende equivalencias durante el pretraining
#    - No necesitamos lowercase manual
#
# 3. **Embeddings contextuales capturan semántica**:
#    - BETO entiende "no tengo apetito" vs "tengo apetito" sin marcadores
#    - La atención captura negación implícitamente
#    - No necesitamos heurísticas como "no_X"
#
# Preprocesamiento conservador para transformers
#
# Estrategia: Mínimo (solo colapsa alargamientos, preserva todo lo demás)
# - BETO se entrenó con texto natural (mayúsculas, tildes, puntuación)
# - WordPiece tokenization maneja variaciones automáticamente
# - Comparación: Rule-based conserva para patterns, TF-IDF normaliza, BETO preserva distribución original

import pandas as pd, re, unicodedata

# Cargar splits unificados desde 02_create_splits.ipynb
dataset_base = pd.read_csv(SPLITS_PATH / 'dataset_base.csv')
train_indices = pd.read_csv(SPLITS_PATH / 'train_indices.csv')['row_id'].values
val_indices = pd.read_csv(SPLITS_PATH / 'val_indices.csv')['row_id'].values

text_col = _guess_text_col(dataset_base)
label_col = _guess_label_col(dataset_base)

print(f"[INFO] Splits: {len(train_indices)} train, {len(val_indices)} val")

# Definir función de limpieza conservadora
RE_MULTI = re.compile(r'(.)\1{2,}')  # Detecta 3+ letras repetidas

def clean_text_trf(s: str) -> str:
    """
    Limpieza CONSERVADORA para transformers (mínimo preprocesamiento).
    
    Aplica ÚNICAMENTE:
    - Normalización NFC (forma canónica de tildes)
    - Colapso de alargamientos (holaaa → holaa)
    - Normalización de espacios
    
    Preserva:
    - Mayúsculas y minúsculas (BETO las usa)
    - Tildes y acentos (parte del vocabulario)
    - Puntuación (señal contextual)
    - Estructura original del texto
    """
    if pd.isna(s):
        return ""
    
    s = str(s).strip()
    s = unicodedata.normalize("NFC", s)  # Normaliza tildes (é = é, no e + ´)
    s = RE_MULTI.sub(r'\1\1', s)         # holaaa → holaa (evita OOV)
    s = re.sub(r"\s+", " ", s).strip()   # Colapsa espacios múltiples
    
    return s

dataset_base['texto_trf'] = dataset_base[text_col].map(clean_text_trf)

df_train = dataset_base[dataset_base['row_id'].isin(train_indices)].copy()
df_val = dataset_base[dataset_base['row_id'].isin(val_indices)].copy()

X_train, y_train = df_train['texto_trf'], df_train[label_col]
X_val, y_val = df_val['texto_trf'], df_val[label_col]

print(f"[INFO] Distribución train: {dict(y_train.value_counts())}")
print(f"[INFO] Distribución val: {dict(y_val.value_counts())}")

[INFO] Splits: 2509 train, 646 val
[INFO] Distribución train: {'depresion': 1745, 'ansiedad': 764}
[INFO] Distribución val: {'depresion': 485, 'ansiedad': 161}
[INFO] Distribución train: {'depresion': 1745, 'ansiedad': 764}
[INFO] Distribución val: {'depresion': 485, 'ansiedad': 161}


## 2) Tokenización y datasets

In [3]:
from datasets import Dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

MODEL = "dccuchile/bert-base-spanish-wwm-cased"  # Español cased
tok = AutoTokenizer.from_pretrained(MODEL)

# Mapeo de etiquetas a IDs numéricos
label2id = {'depresion': 0, 'ansiedad': 1}
id2label = {0: 'depresion', 1: 'ansiedad'}

# CRÍTICO: Agregar columna 'labels' numérica (el Trainer la requiere)
df_train['labels'] = df_train[label_col].map(label2id)
df_val['labels'] = df_val[label_col].map(label2id)

print(f"[INFO] Mapeo de etiquetas: {label2id}")
print(f"[INFO] Distribución train (numérica): {dict(df_train['labels'].value_counts())}")
print(f"[INFO] Distribución val (numérica): {dict(df_val['labels'].value_counts())}")

def preprocess(batch):
    return tok(batch["texto_trf"], truncation=True, padding=False, max_length=256)

# Crear datasets con columnas: texto_trf y labels
train_ds = Dataset.from_pandas(df_train[['texto_trf', 'labels']].reset_index(drop=True)).map(preprocess, batched=True, remove_columns=["texto_trf"])
val_ds = Dataset.from_pandas(df_val[['texto_trf', 'labels']].reset_index(drop=True)).map(preprocess, batched=True, remove_columns=["texto_trf"])

collator = DataCollatorWithPadding(tokenizer=tok)

print(f"\n[INFO] Datasets creados:")
print(f"  Train: {len(train_ds)} ejemplos")
print(f"  Val: {len(val_ds)} ejemplos")
print(f"  Columnas train_ds: {train_ds.column_names}")
print(f"  Columnas val_ds: {val_ds.column_names}")

[INFO] Mapeo de etiquetas: {'depresion': 0, 'ansiedad': 1}
[INFO] Distribución train (numérica): {0: 1745, 1: 764}
[INFO] Distribución val (numérica): {0: 485, 1: 161}


Map:   0%|          | 0/2509 [00:00<?, ? examples/s]

Map:   0%|          | 0/646 [00:00<?, ? examples/s]


[INFO] Datasets creados:
  Train: 2509 ejemplos
  Val: 646 ejemplos
  Columnas train_ds: ['labels', 'input_ids', 'token_type_ids', 'attention_mask']
  Columnas val_ds: ['labels', 'input_ids', 'token_type_ids', 'attention_mask']


## 3) Entrenamiento y evaluación

In [4]:
import evaluate, numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# Mapeo ya definido en celda anterior
# label2id = {'depresion': 0, 'ansiedad': 1}
# id2label = {0: 'depresion', 1: 'ansiedad'}

model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2, id2label=id2label, label2id=label2id)

metric_f1   = evaluate.load("f1")
metric_prec = evaluate.load("precision")
metric_rec  = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "macro_f1":   metric_f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "macro_precision": metric_prec.compute(predictions=preds, references=labels, average="macro")["precision"],
        "macro_recall":    metric_rec.compute(predictions=preds, references=labels, average="macro")["recall"],
    }

args = TrainingArguments(
    output_dir="runs/beto_ad",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    greater_is_better=True,
    seed=42,
    logging_steps=50
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tok,
    data_collator=collator,
    compute_metrics=compute_metrics
)

print("[INFO] Iniciando entrenamiento...")
print(f"  Epochs: {args.num_train_epochs}")
print(f"  Batch size: {args.per_device_train_batch_size}")
print(f"  Learning rate: {args.learning_rate}")

trainer.train()
eval_res = trainer.evaluate()

import pandas as pd
(pd.DataFrame([eval_res]).to_csv(DATA_PATH/'beto_eval.csv', index=False, encoding='utf-8'))
print(f"\n[INFO] Entrenamiento completado")
print(f"  Macro F1: {eval_res['eval_macro_f1']:.4f}")
print(f"  Macro Precision: {eval_res['eval_macro_precision']:.4f}")
print(f"  Macro Recall: {eval_res['eval_macro_recall']:.4f}")
print(f"[INFO] Eval guardada: {DATA_PATH/'beto_eval.csv'}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/25/fy01l91x3gj63g090ghxj7000000gn/T/ipykernel_3805/719104514.py:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/var/folders/25/fy01l91x3gj63g090ghxj7000000gn/T/ipykernel_3805/719104514.py:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


[INFO] Iniciando entrenamiento...
  Epochs: 3
  Batch size: 16
  Learning rate: 2e-05


Epoch,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall
1,0.326700,0.481634,0.694681,0.714829,0.682417
2,0.214900,0.547733,0.704530,0.712809,0.697983
3,0.127700,0.587868,0.736864,0.743670,0.731088



[INFO] Entrenamiento completado
  Macro F1: 0.7369
  Macro Precision: 0.7437
  Macro Recall: 0.7311
[INFO] Eval guardada: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/beto_eval.csv


## 4) Reporte detallado y predicciones

In [5]:
import numpy as np, pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

pred_logits = trainer.predict(val_ds).predictions
pred_ids = pred_logits.argmax(axis=-1)
y_true = df_val["labels"].to_numpy()  # CORREGIDO: usar 'labels' en lugar de 'label'

# Exportables
beto_pred_csv   = DATA_PATH/'beto_predictions.csv'
beto_report_csv = DATA_PATH/'beto_classification_report.csv'
beto_eval_csv   = DATA_PATH/'beto_eval.csv'  # ya creado arriba
beto_cm_csv     = DATA_PATH/'beto_confusion_matrix.csv'

pd.DataFrame(classification_report(y_true, pred_ids, target_names=['depresion','ansiedad'], output_dict=True, zero_division=0)).transpose().to_csv(beto_report_csv, index=True, encoding='utf-8')

cm = confusion_matrix(y_true, pred_ids, labels=[0,1])
pd.DataFrame(cm, index=['true_depresion','true_ansiedad'], columns=['pred_depresion','pred_ansiedad']).to_csv(beto_cm_csv)

# Con textos (útil para análisis de errores)
val_out = df_val.copy()
val_out["y_true"] = val_out["labels"].map({0:"depresion",1:"ansiedad"})
val_out["y_pred"] = [id2label[i] for i in pred_ids]
val_out.to_csv(beto_pred_csv, index=False, encoding="utf-8")

print("[INFO] Exportados:")
print(f"  - Predicciones: {beto_pred_csv}")
print(f"  - Reporte: {beto_report_csv}")
print(f"  - Eval: {beto_eval_csv}")
print(f"  - Matriz: {beto_cm_csv}")

# Mostrar reporte en consola
print("\n" + "="*60)
print("CLASSIFICATION REPORT (BETO)")
print("="*60)
print(classification_report(y_true, pred_ids, target_names=['depresion','ansiedad'], zero_division=0))
print("\nMatriz de Confusión:")
print(pd.DataFrame(cm, index=['true_depresion','true_ansiedad'], columns=['pred_depresion','pred_ansiedad']))

[INFO] Exportados:
  - Predicciones: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/beto_predictions.csv
  - Reporte: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/beto_classification_report.csv
  - Eval: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/beto_eval.csv
  - Matriz: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/beto_confusion_matrix.csv

CLASSIFICATION REPORT (BETO)
              precision    recall  f1-score   support

   depresion       0.86      0.88      0.87       485
    ansiedad       0.62      0.58      0.60       161

    accuracy                           0.81       646
   macro avg       0.74      0.73      0.74       646
weighted avg       0.80      0.81      0.81       646


Matriz de Confusión:
                pred_depresion  pred_ansiedad
true_depresion             429             56
true_ansiedad               68             93


## 5) Análisis de Errores (FP/FN)

In [6]:
# Exportar errores para análisis cualitativo
# Usar val_out que ya tiene y_true/y_pred en formato texto

fp_depresion = val_out[(val_out['y_true'] == 'ansiedad') & (val_out['y_pred'] == 'depresion')].copy()
fp_depresion['error_type'] = 'FP_depresion'

fn_depresion = val_out[(val_out['y_true'] == 'depresion') & (val_out['y_pred'] == 'ansiedad')].copy()
fn_depresion['error_type'] = 'FN_depresion'

fp_ansiedad = val_out[(val_out['y_true'] == 'depresion') & (val_out['y_pred'] == 'ansiedad')].copy()
fp_ansiedad['error_type'] = 'FP_ansiedad'

fn_ansiedad = val_out[(val_out['y_true'] == 'ansiedad') & (val_out['y_pred'] == 'depresion')].copy()
fn_ansiedad['error_type'] = 'FN_ansiedad'

beto_fp_dep_csv = DATA_PATH / 'beto_fp_depresion.csv'
beto_fn_dep_csv = DATA_PATH / 'beto_fn_depresion.csv'
beto_fp_ans_csv = DATA_PATH / 'beto_fp_ansiedad.csv'
beto_fn_ans_csv = DATA_PATH / 'beto_fn_ansiedad.csv'

fp_depresion[['texto_trf', 'y_true', 'y_pred', 'error_type']].to_csv(beto_fp_dep_csv, index=False, encoding='utf-8')
fn_depresion[['texto_trf', 'y_true', 'y_pred', 'error_type']].to_csv(beto_fn_dep_csv, index=False, encoding='utf-8')
fp_ansiedad[['texto_trf', 'y_true', 'y_pred', 'error_type']].to_csv(beto_fp_ans_csv, index=False, encoding='utf-8')
fn_ansiedad[['texto_trf', 'y_true', 'y_pred', 'error_type']].to_csv(beto_fn_ans_csv, index=False, encoding='utf-8')

print("[INFO] Análisis de errores exportado:")
print(f"  FP Depresión: {len(fp_depresion)} casos → {beto_fp_dep_csv.name}")
print(f"  FN Depresión: {len(fn_depresion)} casos → {beto_fn_dep_csv.name}")
print(f"  FP Ansiedad:  {len(fp_ansiedad)} casos → {beto_fp_ans_csv.name}")
print(f"  FN Ansiedad:  {len(fn_ansiedad)} casos → {beto_fn_ans_csv.name}")

[INFO] Análisis de errores exportado:
  FP Depresión: 68 casos → beto_fp_depresion.csv
  FN Depresión: 56 casos → beto_fn_depresion.csv
  FP Ansiedad:  56 casos → beto_fp_ansiedad.csv
  FN Ansiedad:  68 casos → beto_fn_ansiedad.csv
